In [14]:
import pickle
import numpy as np
from tqdm import tqdm
from functools import partial
import pandas as pd
from konlpy.tag import Komoran
import os
from gensim.models import Word2Vec

In [2]:
preprocessed = pickle.load(open('preprocessed.pkl', 'rb'))

In [3]:
max_n = 5
total_data = [[], [], [], [], []]
for i in range(1, max_n + 1):
    file_name = 'data_256_cat%s.pkl' %i
    loaded_data = pickle.load(open(file_name, 'rb'))

    _image_data, _sentence_data, _sentence_tags, _hashtag_data, _metadata = loaded_data[1]

    _sentence_tags = np.array(_sentence_tags)
    _hashtag_data = np.array(_hashtag_data)
    
    total_data[0].append(_image_data)
    total_data[1].append(_sentence_data)
    total_data[2].append(_sentence_tags)
    total_data[3].append(_hashtag_data)
    total_data[4].append(_metadata)

In [4]:
_image_data = np.concatenate(total_data[0])
_sentence_data = np.concatenate(total_data[1])
_sentence_tags = np.concatenate(total_data[2])
_hashtag_data = np.concatenate(total_data[3])
_metadata = pd.concat(total_data[4])

In [5]:
np.random.seed(1234)
n_valid = 700
n_train = _image_data.shape[0] - n_valid
n_total = n_train + n_valid
indices = np.random.choice(np.arange(_image_data.shape[0]), n_total, replace=False)
train_idx = indices[:n_train]
valid_idx = indices[n_train:]

In [6]:
image_data = _image_data[train_idx]
sentence_data = _sentence_data[train_idx]
sentence_tags = _sentence_tags[train_idx]
hashtag_data = _hashtag_data[train_idx]
metadata = _metadata.iloc[train_idx]

In [162]:
image_valid = _image_data[valid_idx]
sentence_valid = _sentence_data[valid_idx]
sentence_tags_valid = _sentence_tags[valid_idx]
hashtag_valid = _hashtag_data[valid_idx]
metadata_valid = _metadata.iloc[valid_idx]

In [66]:
komoran = Komoran()
w2v_tags = [[t[0] for t in komoran.pos(s)] for s in tqdm(preprocessed['clean_text'])]

100%|██████████| 867737/867737 [29:20<00:00, 492.90it/s] 


In [67]:
# pickle.dump(w2v_tags, open('w2v_tags.pkl', 'wb'))

In [65]:
w2v_tags = pickle.load(open('w2v_tags.pkl', 'rb'))

for tags in w2v_tags:
    tags.insert(0, '\t')
    tags.append('\n')

In [198]:
new_w2v = Word2Vec(
    w2v_tags,
    workers=os.cpu_count(),
    size=200,
    window=5,
    min_count=50,
    sample=1e-3
).wv

In [199]:
new_w2v.most_similar('\n')

[('히히', 0.4601551294326782),
 ('그럼', 0.4597155749797821),
 ('근데', 0.45525985956192017),
 ('증말', 0.454887330532074),
 ('헤헤', 0.4441181421279907),
 ('그래도', 0.3884080648422241),
 ('그나저나', 0.3708135187625885),
 ('얼른', 0.35448527336120605),
 ('그래야', 0.35260123014450073),
 ('헿', 0.3460230231285095)]

In [200]:
print(image_data.shape)
print(sentence_data.shape)
print(sentence_tags.shape)
print(hashtag_data.shape)
print(metadata.shape)

(34300, 256, 256, 3)
(34300,)
(34300,)
(34300,)
(34300, 15)


In [201]:
komoran = Komoran()
all_tags = np.array([[t[0] for t in komoran.pos(s)] for s in tqdm(sentence_data)])

train_indices = [i for i,n in enumerate(map(len, all_tags)) if 0 < n <= 100]
train_imgs = image_data[train_indices]
train_tags = all_tags[train_indices]
train_sens = sentence_data[train_indices]

100%|██████████| 34300/34300 [01:01<00:00, 561.20it/s] 


In [202]:
valid_imgs = image_valid
valid_tags = np.array([[t[0] for t in komoran.pos(s)] for s in tqdm(sentence_valid)])
valid_sens = sentence_valid

100%|██████████| 700/700 [00:01<00:00, 440.47it/s] 


In [203]:
# len_dict = {}
# for i in map(len, all_tags):
#     if i not in len_dict:
#         len_dict[i] = 1
#     else:
#         len_dict[i] += 1
        
# for i, n in sorted(len_dict.items()):
#     print(i, '\t', n)

In [204]:
max_seq_length = max(map(len, train_tags))
gensim_data = np.zeros(shape=(len(train_tags), max_seq_length, 200))

for i, tags in enumerate(train_tags):
    for j, tag in enumerate([t for t in tags if t in new_w2v]):
        gensim_data[i, j] = new_w2v[tag]
        
gensim_valid = np.zeros(shape=(len(valid_tags), max_seq_length, 200))

for i, tags in enumerate(valid_tags):
    for j, tag in enumerate([t for t in tags if t in new_w2v][:100]):
        gensim_valid[i, j] = new_w2v[tag]

In [205]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [247]:
n_hidden = 256
n_output = 200

def generator(images, labels=None):
    with tf.variable_scope('generator'):
        conv1 = tf.layers.conv2d(images, 16, [5,5], strides=2, padding='SAME')
        conv2 = tf.layers.conv2d(conv1, 8, [5,5], strides=2, padding='SAME')
        conv3 = tf.layers.conv2d(conv2, 4, [5,5], strides=2, padding='SAME')
        reshaped = tf.reshape(conv3, shape=(-1, 128, 32))
        rnn_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
        rnn_cell = tf.nn.rnn_cell.DropoutWrapper(rnn_cell, output_keep_prob=0.8)
        outputs, states = tf.nn.dynamic_rnn(rnn_cell, reshaped, dtype=tf.float32)
        w2v_outputs = tf.layers.dense(outputs[:max_seq_length], n_output)
    return w2v_outputs

def discriminator(inputs, labels=None, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
        rnn_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
        rnn_cell = tf.nn.rnn_cell.DropoutWrapper(rnn_cell, output_keep_prob=0.8)
        outputs, states = tf.nn.dynamic_rnn(rnn_cell, inputs, dtype=tf.float32)
        rnn_output = outputs[:, -1]
        output = tf.layers.dense(rnn_output, 1, activation=None)
    return output

In [251]:
G

<tf.Tensor 'generator/dense/BiasAdd:0' shape=(?, 128, 200) dtype=float32>

In [249]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 256, 256, 3])
Y = tf.placeholder(tf.float32, [None, max_seq_length, 200])

G = generator(X)
D_real = discriminator(Y)
D_gene = discriminator(G, reuse=True)

loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [252]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [253]:
batch_size = 1800
n_epoch = 100
n_batch = train_imgs.shape[0] // batch_size

for epoch in range(n_epoch):
    total_loss_D = 0
    total_loss_G = 0
    for batch in tqdm(range(n_batch)):
        batch_img = train_imgs[batch*batch_size:(batch+1)*batch_size]
        batch_tag = gensim_data[batch*batch_size:(batch+1)*batch_size]
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X:batch_img, Y:batch_tag})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={X:batch_img})
        
        total_loss_D += loss_val_D
        total_loss_G += loss_val_G
        
#         print('[Epoch: %s Batch: %s] loss_D = %.3f, loss_G = %.3f' %(epoch+1, batch+1, loss_val_D, loss_val_G))
        
    _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X:valid_imgs, Y:gensim_valid})
    _, loss_val_G = sess.run([train_G, loss_G], feed_dict={X:valid_imgs})
    
    print('[Epoch: %s] loss_D = %.3f, loss_G = %.3f, loss_valid_D = %.3f, loss_valid_G = %.3f' %(epoch+1, total_loss_D / n_batch, total_loss_G / n_batch, loss_val_D, loss_val_G))

  0%|          | 0/18 [00:00<?, ?it/s]

[Epoch: 1] loss_D = 1.444, loss_G = 2.359, loss_valid_D = 1.722, loss_valid_G = 1.299


  0%|          | 0/18 [00:00<?, ?it/s]

[Epoch: 2] loss_D = 0.232, loss_G = 3.564, loss_valid_D = 0.075, loss_valid_G = 4.968


  0%|          | 0/18 [00:00<?, ?it/s]

[Epoch: 3] loss_D = 0.037, loss_G = 4.361, loss_valid_D = 0.100, loss_valid_G = 3.674


  0%|          | 0/18 [00:00<?, ?it/s]

[Epoch: 4] loss_D = 0.048, loss_G = 3.972, loss_valid_D = 0.082, loss_valid_G = 4.606


 61%|██████    | 11/18 [00:34<00:21,  3.11s/it]


KeyboardInterrupt: 

In [219]:
results = sess.run(G, feed_dict={X: train_imgs[:10]})

for i, result in enumerate(results):
    print(train_sens[i][1:-1])
    print(' '.join([new_w2v.most_similar([v])[0][0] for v in result]))

실버커플링중 언벨런스느낌과 독특한면이 독보이는 디자인입니다 남자반지는 유광무광으로 고급스러움을 살림 커플링입니다 홈페이지 카카오톡문의 고객센터 대한민국에서 제일 저렴한 쥬얼리쇼핑몰 검색창에 피카커플링을 검색해주세요 오프라인매장 종로가역 번출구 앞 조흥상가층
도록 며 며 도록 며 도록 ㅂ니다 도록 ㅂ니다 도록 도록 며 ㅂ니다 며 도록 도록 며 도록 도록 도록 도록 도록 ㅂ니다 ㅂ니다 며 ㅂ니다 ㅂ니다 ㅂ니다 도록 도록 ㅂ니다 도록 ㅂ니다 며 도록 도록 며 도록 ㅂ니다 ㅂ니다 ㅂ니다 도록 도록 도록 도록 도록 도록 며 도록 도록 도록 도록 며 도록 ㅂ니다 도록 도록 도록 도록 며 며 며 ㅂ니다 며 며 ㅂ니다 도록 도록 ㅂ니다 ㅂ니다 ㅂ니다 며 도록 ㅂ니다 도록 며 ㅂ니다 ㅂ니다 ㅂ니다 ㅂ니다 ㅂ니다 며 며 ㅂ니다 도록 며 도 며 ㅂ니다 며 도록 ㅂ니다 며 며 도록 도록 ㅂ니다 ㅂ니다 ㅂ니다 도록
우지혁 하이
며 ㅂ니다 ㅂ니다 도록 며 도록 ㅂ니다 며 며 며 ㅂ니다 도록 ㅂ니다 며 ㅂ니다 도록 도록 ㅂ니다 며 ㅂ니다 도록 ㅂ니다 ㅂ니다 도록 ㅂ니다 ㅂ니다 ㅂ니다 도록 ㅂ니다 ㅂ니다 며 ㅂ니다 도록 도록 ㅂ니다 ㅂ니다 도록 며 ㅂ니다 며 며 ㅂ니다 며 며 도록 도록 ㅂ니다 도록 며 도록 ㅂ니다 ㅂ니다 도록 며 ㅂ니다 ㅂ니다 ㅂ니다 ㅂ니다 도록 며 며 도록 며 ㅂ니다 도록 도록 도록 며 ㅂ니다 도록 도록 ㅂ니다 도록 ㅂ니다 도록 ㅂ니다 ㅂ니다 도록 ㅂ니다 ㅂ니다 도록 며 ㅂ니다 도록 ㅂ니다 도록 며 도록 ㅂ니다 ㅂ니다 도록 ㅂ니다 며 며 도록 며 ㅂ니다 도록 며 도록
어흥 눈뜨면 아침이고 월요일인가하면 벌써 주말이고 월초인가하면 어느새 월말을 향하고 세월이 빠른건지 내가 급한건지 아니면 삶이 짧아진건지 마음속의 나는 그대로인데 거울속의 나는 어느덧 늙어있네
며 며 도록 도록 도록 며 도록 ㅂ니다 도록 ㅂ니다 도록 ㅂ니다 ㅂ니다 며 며 ㅂ니다 며 며 며 며 도록 도록 도록 ㅂ니다 ㅂ니다 ㅂ니다 ㅂ니다 ㅂ니다 ㅂ니다 ㅂ니다 며 도록 ㅂ니다 ㅂ니다 며 며 며 ㅂ니다 도록 ㅂ

In [79]:
from PIL import Image
import matplotlib.pyplot as plt

In [98]:
def translate(image):
    prediction = tf.argmax(outputs2, 2)
    dec_inp = np.zeros(shape=(1, max_seq_length, dict_len), dtype='float32')
    dec_inp[0,:,output_dict['\t']] = 1.
    # dec_out = np.eye(dict_len)[dec_out]
    result = sess.run(prediction, 
                      feed_dict={inputs: [image],
                                 dec_inputs: dec_inp})
    decoded = [output_chars[i] for i in result[0]]
    end = decoded.index('\n') if '\n' in decoded else len(decoded)
    translated = ''.join(decoded[:end])
    return translated

In [103]:
translate(image_data[50])

'오<UNK><UNK>'

In [80]:
start_idx, end_idx = 50, 100

results = sess.run(outputs2, feed_dict={inputs:image_data[start_idx:end_idx]})
print('정답 예측')

for l, p, img in zip(y_train[start_idx:end_idx], l_label[np.argmax(results, axis=1)], image_data[start_idx:end_idx]):
    print([l==p, l, p])
    plt.imshow(img)
    plt.show()

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,256,2069]
	 [[node Placeholder_1 (defined at <ipython-input-70-42deb1a72a5c>:19) ]]
	 [[generation_dec/dense/BiasAdd/_167]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,256,2069]
	 [[node Placeholder_1 (defined at <ipython-input-70-42deb1a72a5c>:19) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Placeholder_1':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/traitlets/config/application.

In [52]:
start_idx, end_idx = 0, 50

results = sess.run(outputs2, feed_dict={inputs:image_valid[start_idx:end_idx]})
print('정답 예측')
[[l==p, l, p] for l, p in zip(y_train[start_idx:end_idx], l_label[np.argmax(results, axis=1)])]

정답 예측


[[False, '애완동물', '여행'],
 [False, '일상', '여행'],
 [False, '애완동물', '여행'],
 [False, '셀카', '패션'],
 [False, '여행', '셀카'],
 [False, '여행', '연애'],
 [False, '애완동물', '음식'],
 [False, '음식', '여행'],
 [False, '패션', '여행'],
 [True, '애완동물', '애완동물'],
 [False, '일상', '애완동물'],
 [False, '여행', '애완동물'],
 [False, '패션', '연애'],
 [True, '패션', '패션'],
 [False, '음식', '여행'],
 [False, '여행', '음식'],
 [False, '음식', '여행'],
 [False, '패션', '애완동물'],
 [False, '음식', '셀카'],
 [False, '애완동물', '연애'],
 [False, '일상', '음식'],
 [True, '여행', '여행'],
 [False, '음식', '패션'],
 [False, '연애', '셀카'],
 [False, '셀카', '음식'],
 [False, '음식', '패션'],
 [False, '애완동물', '연애'],
 [False, '패션', '일상'],
 [False, '셀카', '애완동물'],
 [False, '일상', '패션'],
 [False, '음식', '여행'],
 [True, '패션', '패션'],
 [False, '패션', '일상'],
 [False, '일상', '연애'],
 [True, '연애', '연애'],
 [False, '애완동물', '패션'],
 [False, '일상', '연애'],
 [False, '애완동물', '패션'],
 [False, '애완동물', '패션'],
 [False, '연애', '여행'],
 [False, '셀카', '여행'],
 [True, '음식', '음식'],
 [False, '여행', '애완동물'],
 [True, '애완동물', '애완동물'],
 [Fal